# IMPORT DATA, PACKAGES, AND FUNCTIONS

## import the relevant functions and packages

In [1]:
import pandas as pd
import numpy as np
from test_case_generator_functions import alphabet, num_of_alphabet, rand_letter

# TEST CASE TYPE REFERENCE INFORMATION

In [2]:
uid = 'UID-292' 
theme = 'Names where name parts are Modified'
category = 'Typos'
sub_category = '> 2 Typos same name part - Adjacent, at beginning'
entity_type = 'Entity'

## download the OFAC list from the web

In [3]:
ofac_list_download = pd.read_csv('https://www.treasury.gov/ofac/downloads/sdn.csv', header=None)
ofac_list_download

,0,1,2,3,4,5,6,7,8,9,10,11
0,36,AEROCARIBBEAN AIRLINES,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
2,306,BANCO NACIONAL DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,a.k.a. 'BNC'.
3,424,BOUTIQUE LA MAISON,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
4,475,CASA DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
...,...,...,...,...,...,...,...,...,...,...,...,...
10892,39438,"NEJAT, Hossein",individual,IRGC] [IFSR] [IRAN-HR,IRGC Brigadier General,-0-,-0-,-0-,-0-,-0-,-0-,"DOB Mar 1955 to Mar 1956; POB Shiraz, Iran; na..."
10893,39439,"SAJEDINIA, Hossein",individual,IRAN-HR,Deputy Operations Commander,-0-,-0-,-0-,-0-,-0-,-0-,"DOB 21 Mar 1962 to 20 Apr 1962; POB Isfahan, I..."
10894,39440,"RAHIMI, Hossein",individual,IRAN-HR,Police chief of Tehran,-0-,-0-,-0-,-0-,-0-,-0-,DOB Mar 1963 to Mar 1964; POB Markazi Province...
10895,39441,"JAVANI, Yadollah",individual,IRGC] [IFSR] [IRAN-HR,Deputy Political Commander,-0-,-0-,-0-,-0-,-0-,-0-,"DOB 1962; POB Isfahan, Iran; nationality Iran;..."


In [4]:
ofac_list = ofac_list_download[[0,1,2]]
ofac_list.columns = ['uid', 'name', 'entity_type']
ofac_list

,uid,name,entity_type
0,36,AEROCARIBBEAN AIRLINES,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-
2,306,BANCO NACIONAL DE CUBA,-0-
3,424,BOUTIQUE LA MAISON,-0-
4,475,CASA DE CUBA,-0-
...,...,...,...
10892,39438,"NEJAT, Hossein",individual
10893,39439,"SAJEDINIA, Hossein",individual
10894,39440,"RAHIMI, Hossein",individual
10895,39441,"JAVANI, Yadollah",individual


# FILTER FOR THE REQUIREMENTS OF THE TEST CASE TYPE

## filter for the requirements of the specific test case type

In [5]:
ofac_list_filtered = ofac_list[(ofac_list.entity_type == '-0- ')] # only evaluate entities
ofac_list_filtered

,uid,name,entity_type
0,36,AEROCARIBBEAN AIRLINES,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-
2,306,BANCO NACIONAL DE CUBA,-0-
3,424,BOUTIQUE LA MAISON,-0-
4,475,CASA DE CUBA,-0-
...,...,...,...
10884,39282,SOPHYCHEM HK LIMITED,-0-
10885,39283,CLARA SHIPPING LLC,-0-
10886,39356,NEW EASTERN SHIPPING CO LTD,-0-
10888,39358,ANFASAR TRADING S PTE. LTD.,-0-


## randomly choose 10 rows

In [6]:
while True:
    ofac_list_sampled = ofac_list_filtered.sample(n = 10)
    cnt = [0] * 10
    for ind, name in enumerate(ofac_list_sampled['name']):
        split_name = name.upper().split(' ')
        
        if all(i in alphabet() for i in list(split_name[0][:3])) & (len(split_name[0]) > 3) & (len(' '.join(split_name)) > 10):
            cnt[ind] += 1

    if all(i == 1 for i in cnt):
        break

ofac_list_sampled

,uid,name,entity_type
7149,27008,GENERAL DIRECTORATE OF MILITARY COUNTERINTELLI...,-0-
5231,22264,BARLY OFF-SHORE,-0-
344,7287,GANADERIAS BILBAO LTDA.,-0-
7243,27236,GLOBAL ENERGY COMPANY S.A.S.,-0-
2997,15543,PRESS SUPERVISORY BOARD,-0-
7828,29270,MURUJ CHAM INVESTMENT AND TOURISM GROUP,-0-
420,7578,CIA. ANDINA DE EMPAQUES LTDA.,-0-
10879,39277,IRAN CHEMICAL INDUSTRIES INVESTMENT COMPANY PU...,-0-
6366,25458,EIGHTH OCEAN GMBH & CO. KG,-0-
7045,26808,CEVICHERIA LA CONCHA,-0-


# CREATE THE TEST CASES

## create blank final test cases table

In [7]:
final_test_cases = pd.DataFrame(columns=['UID', 'Theme','Category','Sub-category','Entity-Type','Test Case ID' , 'OFAC List UID', 'Original Name','Test Case Name'])
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name


## run loop to generate the test cases

In [8]:
for index, row in ofac_list_sampled.iterrows():
    original_name = row['name'].upper()
    split_original_name = original_name.split(' ') # split name into word
    
    final_test_name = split_original_name.copy()
    
    replace_word = split_original_name[0]
    replace_word_list = list(replace_word)

    if num_of_alphabet(replace_word) == 3:
        random_length = 3
    else:
        random_length = np.random.randint(3, num_of_alphabet(replace_word))

    for i in range(random_length):
        while replace_word_list[i] == replace_word[i]: # rerandom if no letter has been replaced
            replace_word_list[i] = rand_letter() # replace letter

    final_test_name[final_test_name.index(replace_word)] = ''.join(replace_word_list)
    
    final_test_name = ' '.join(final_test_name)
    
    final_test_cases.loc[len(final_test_cases)] = [uid, theme, category, sub_category, entity_type, uid + ' - ' + str(index), row['uid'], row['name'], final_test_name] # append to the dataframe

final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name
0,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 7149,27008,GENERAL DIRECTORATE OF MILITARY COUNTERINTELLI...,JSIRRAL DIRECTORATE OF MILITARY COUNTERINTELLI...
1,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 5231,22264,BARLY OFF-SHORE,LUKLY OFF-SHORE
2,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 344,7287,GANADERIAS BILBAO LTDA.,EYSGQMFQAS BILBAO LTDA.
3,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 7243,27236,GLOBAL ENERGY COMPANY S.A.S.,VBCBAL ENERGY COMPANY S.A.S.
4,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 2997,15543,PRESS SUPERVISORY BOARD,HIHSS SUPERVISORY BOARD
5,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 7828,29270,MURUJ CHAM INVESTMENT AND TOURISM GROUP,TZUWJ CHAM INVESTMENT AND TOURISM GROUP
6,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 420,7578,CIA. ANDINA DE EMPAQUES LTDA.,KZP. ANDINA DE EMPAQUES LTDA.
7,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 10879,39277,IRAN CHEMICAL INDUSTRIES INVESTMENT COMPANY PU...,WIKN CHEMICAL INDUSTRIES INVESTMENT COMPANY PU...
8,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 6366,25458,EIGHTH OCEAN GMBH & CO. KG,FOQCTH OCEAN GMBH & CO. KG
9,UID-292,Names where name parts are Modified,Typos,"> 2 Typos same name part - Adjacent, at beginning",Entity,UID-292 - 7045,26808,CEVICHERIA LA CONCHA,LCHXNFJDIA LA CONCHA
